## Základné nastavenia

Importujeme potrebné knižnice. Nasledujúci odstavec by mal byť spustený ako prvý predtým ako sa budú používať importované objekty `pd`, `np`, `plt` a `sns`.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# nastavíme zobrazovanie grafov priamo v odstavcoch zápisníka
%matplotlib inline
# inicializujeme knižnicu seaborn
sns.set()

## Možnosti pre načítavanie a zápis dát

Knižnica `pandas` podoruje čítanie a zápis dát z rôznych formátov, ako napr. z textových CSV súborov, alebo súborov programu Microsoft Excel. Pre každý z podporovaných formátov je možné zmeniť nastavenia, ako napr. znak pre oddeľovať hodnôt, desatinnú čiarku, alebo či súbor obsahuje na prvom riadku hlavičku s názvami stĺpcov/atribútov.

Viac o možnostiach načítavania a zápisu dát v knižnici `pandas` si môžete prečítať [tu](https://pandas.pydata.org/pandas-docs/stable/reference/io.html) (v angličtine).

In [ ]:
# pre CSV súbory viete zmeniť napr. oddeľovač hodnôt (parameter `delimiter`) a znak desatinnej čiarky (`decimal`)
# ak sa v súbore nenacháza hlavička s názvami atribútov na prvom riadku (parameter `header=None`), atribúty je možné pomenovať
# v parametre `names`
data = pd.read_csv("../data/iris.csv", delimiter=";", decimal=",", header=None, names=["sepal_length", "sepal_width", "petal_length", "petal_width", "species"])
data.head()

In [ ]:
# podobne môžete načítať dáta z Excel súboru (štandardne sa načíta dátová tabuľka z prvého zošitu)
data = pd.read_excel("../data/iris.xlsx")

Pre zápis dát do súboru použite priamo metódy dátového rámca.

In [ ]:
# štandardne sa do súboru zapíše aj stĺpec s indexmi riadkov, ak chceme zapísať iba atribúty bez indexu, nastavíme `index=False`
data.to_csv("../data/iris_processed.csv", index=False)

In [ ]:
# pri zápise si môžeme zvoliť, ktoré atribúty sa zapíšu
data.to_excel("../data/iris_processed.xlsx", index=False, columns=["sepal_length", "sepal_width"])

## Spájanie tabuliek

Pre základné spájanie dát slúži metóda `concat`, ktorá umožňuje spojiť dáta po riadkoch, alebo po stĺpcoch zarovnaných podľa indexov jednotlivých riadkov. Knižnica `pandas` ďalej umožňuje spájanie tabuliek pomocou kľúčov podobne ako je to v relačných databázach pomocou jazyka SQL.

In [ ]:
# najprv si načítame a predspracujeme dáta s ktorými budeme pracovať: World Happiness Report z rokoch 2015 a 2016
data_2015 = pd.read_csv("../data/2015.csv") # dáta za rok 2015
data_2015 = data_2015[["Country", "Rank", "Score"]] # vyberieme iba stĺpce `Country`, `Rank` a `Score`
# stĺpce `Rank` a `Score` si premenujeme na `Rank 2015` a `Score 2015`
data_2015 = data_2015.rename(columns={"Rank":"Rank 2015", "Score":"Score 2015"})
data_2015.head()

In [ ]:
# podobne si predspracujeme dáta za rok 2016
data_2016 = pd.read_csv("../data/2016.csv")
data_2016 = data_2016[["Country", "Rank", "Score"]]
data_2016 = data_2016.rename(columns={"Rank":"Rank 2016", "Score":"Score 2016"})
data_2016.head()

In [ ]:
# ak použije na spájanie metódu `concat` podľa stĺpcov (`axis=1`), dáta sa spoja podľa poradia v riadkoch
# ak sa odlišuje počet riadkov v spájaných tabuľkách, dáta sa doplnia chýbajúcimi hodnotami
data_all = pd.concat([data_2015, data_2016], axis=1)
data_all.tail()

In [ ]:
# pre spájanie tabuliek podľa kľúčov použijeme metódu `merge`
# v základných parametroch nastavíme spájané dáta (pravú a ľavú tabuľku) a atribúty, ktoré sa použijú ako kľúč
data_all = pd.merge(left=data_2015, right=data_2016, left_on="Country", right_on="Country")
data_all.tail()

In [ ]:
# skontrolujeme si počet riadkov a chýbajúcich hodnôt v spojenej tabuľke
print(len(data_2015), len(data_2016), len(data_all))
data_all.isna().sum()

Štandardne sa pri spájaní do výslednej tabuľky zahrnú iba tie riadky, ktorých kľúč sa nachádzal v oboch spájaných tabuľkách (tzn. metóda `inner`). Ďalšie možnosti sú podobné ako v jazyku SQL:

| Metóda merge | SQL | Popis |
| --- | --- | --- |
| left | LEFT OUTER JOIN | Iba kľúče z ľavej tabuľky |
| right | RIGHT OUTER JOIN | Iba kľúče z pravej tabuľky |
| outer | FULL OUTER JOIN | Vyber dáta ak majú kľúč v pravej alebo v ľavej tabuľke (zjednotenie) |
| inner | INNER JOIN | Vyber dáta ak majú kľúč aj v pravej aj v ľavej tabuľke (prienik) |


In [ ]:
# spájanie podľa kľúčov ľavej tabuľky (ak sa kľúč nenachádza v pravej tabuľke, doplnia sa chýbajúce hodnoty)
data_all = pd.merge(left=data_2015, right=data_2016, left_on="Country", right_on="Country", how="left")
print(len(data_2015), len(data_2016), len(data_all))
data_all.isna().sum()

In [ ]:
# zjednotenie hodnôt z oboch tabuliek
data_all = pd.merge(left=data_2015, right=data_2016, left_on="Country", right_on="Country", how="outer")
print(len(data_2015), len(data_2016), len(data_all))
data_all.isna().sum()

## Normalizovanie dát

Pri niektorých metódach analýzy dát je potrebné porovnávať hodnoty atrubútov v rovnakých rozsahoch, tzn. dáta je potrebné *normalizovať*. Jedna zo základných metód je prevod do zvoleného intervalu (najčastejšie 0-1). Pre normalizovanie dát budeme používať objekty z knižnice `sklearn`, ktorá obsahuje viacero pokročilých metód a algoritmov pre analýzu dát.

In [ ]:
# naimportujeme si potrebné objekty z knižnice `sklearn`
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# načítame si dátovú množinu `iris`
iris = pd.read_csv("../../6/data/iris.csv")
# do premennej `iris_data` si uložíme číselné atribúty
iris_data = iris[["sepal_length", "sepal_width", "petal_length", "petal_width"]]
# označenie druhov (triedy) si uložíme do premennej `iris_data`
iris_labels = iris["species"]

In [ ]:
# pre normalizáciu dát do intervalu 0-1 použijeme objekt MinMaxScaler
# vytvoríme si objekt normalizácie
min_max = MinMaxScaler()
# transformujeme dáta, výsledok je číselné pole knižnice `numpy`, ktoré nazad prevedieme na dátový rámec `pandas`
norm_array = min_max.fit_transform(iris_data)
# vytvoríme nový dátový rámec z `numpy` poľa, názvy stĺpcov zachováme z pôvodnej množiny `iris_data`
iris_data_norm = pd.DataFrame(norm_array, columns=iris_data.columns)
iris_data_norm.describe()

*Štandardizácia* je špeciálna metóda normalizácie dát, kedy sa dáta prepočítajú tak, aby mali 0 strednú hodnotu (priemer) a 1 štandardnú odchýlku (resp. rozptyl). Pri prepočte sa od každej pôvodnej hodnoty odpočíta priemer pre celý stĺpec a výsledok sa vydelí štandardnou odchýlkou pôvodných dát.

In [ ]:
standard = StandardScaler()
std_array = standard.fit_transform(iris_data)
iris_data_std = pd.DataFrame(norm_array, columns=iris_data.columns)
iris_data_std.describe()

In [ ]:
# vykreslíme si histogram pôvodných, normalizovaných a štandardizovaných dát pre atribút `sepal_length`
# vytvoríme si obrázok rozdelený na 1 riadok a 3 stĺpce, v každom stĺpci zobrazíme jeden histogram
fig, axis = plt.subplots(1, 3)
pl = iris_data["sepal_length"].hist(ax=axis[0]) # `ax` parameter určuje v ktorej časti obrázka sa graf zobrazí
pl = iris_data_norm["sepal_length"].hist(ax=axis[1])
pl = iris_data_std["sepal_length"].hist(ax=axis[2])

## Vizualizácia dát pomocou metódy hlavných komponentov

Vizualizácia je veľmi užitočný nástroj pri analýze dát. Hlavným obmedzením pri vizualizácii dát je to, že je možné zobraziť iba dvoj a trojrozmerné grafy, tzn. naraz si v jednom grafe vieme zobraziť iba závislosti medzi 2-3 atribútmi (ďalšie atribúty môžeme použiť pre vykreslenie ďalších vizuálnych aspektov ako je napr. farba alebo veľkosť bodov a pod.)

Ak si chceme naraz vizualizovať dáta s väčším počtom atribútov, musíme dátovú množinu premietnuť do menej rozmerného 2, alebo 3 rozmerného priestoru. Jednou z metód projekcie dát je metóda hlavných komponentov (*Principal Component Analysis* - PCA), ktorá sa snaží nájsť také premietnutie hodnôt atribútov, ktoré zachová čo najviac vzájomných vzťahov z pôvodných dát. Metóda PCA vypočíta z pôvodných atribútov nové atribúty - tzv. *hlavné komponenty*, ktoré sú usporiadané podľa toho, koľko informácií z pôvodných dát sa do nich premietne. Pre vizualizáciu si vypočítame a zobrazíme závislosť dvoch najdôležitejších komponentov, aby sme zobrazili čo najviac pôvodných vzťahov v dátach.

In [ ]:
# z knižnice `sklearn` si naimportujeme PCA metódu
from sklearn.decomposition import PCA

# vypočítame iba dva najdôležitejšie hlavné komponenty
pca = PCA(n_components=2)
pca_array = pca.fit_transform(iris_data_norm)
# výsledok je `numpy` číselné pole, ktoré si skonvertujeme na dátový rámec `pandas`
# stĺpce si pomenujeme `komponent_1` a `komponent_2`
iris_data_pca = pd.DataFrame(pca_array, columns=["komponent_1", "komponent_2"])

# zobrazíme si ako vyzerajú príklady po premietnutí, hodnoty komponentov už nevieme jednoducho interpretovať
iris_data_pca.head()

In [ ]:
# zobrazíme si, koľko informácií (variancie) v pôvodných dátach je premietnutých do jednotlivých komponentov
pca.explained_variance_ratio_

In [ ]:
# spočítame si percentuálne koľko informácií je celkovo v prvých dvoch hlavných komponentoch, ktoré vizualizujeme
print("{0:.4f}".format(np.sum(pca.explained_variance_ratio_) * 100))

In [ ]:
# spojíme si premietnuté dáta s pôvodnými označeniami príkladov a zobrazíme si dáta na X-Y grafe
# farbu vykreslíme podľa priradenie do jednotlivých druhov rastlín
iris_pca = pd.concat([iris_data_pca, iris_labels], axis=1)
pl = sns.scatterplot(data=iris_pca, x="komponent_1", y="komponent_2", hue="species")

## Úlohy

### Úloha 8.1

Načítajte dáta World Happiness Record 2016 a 2017.

* Vyberte iba atribúty `Country`, `Score` a `Rank`.
* Spojte tabuľky podľa názvu štátu tak aby tam boli všetky hodnoty z oboch tabuliek. Atribúty `Score` a `Rank` vhodne premenujte podľa roku.
* Pridajte nový atribút `Diff` ktorého hodnoty budú rovné `Score 2017` - `Score 2016`
* Zobrazte histogram atribútu `Diff`.

V roku 2017 je viac štátov, ktoré sa zlepšili, alebo zhoršili?

### Úloha 8.2

Načítajte dáta World Happiness Record 2017.

* Vyberte do tabuľky iba základné faktory (`GDP`, `Family`, `Health`, `Freedom`, `Trust`, `Generosity`).
* Normalizujte dáta do intervalu 0-1.
* Pridajte do normalizovanej tabuľky názvy štátov a regiónov.
* Uložte dáta do Excel súboru.

### Úloha 8.3

Načítajte predspracované dáta z predchádzajúcej úlohy.

* Vypočítajte dve hlavné komponenty.
* Vizualizujte hlavné komponenty na X-Y grafe, ako farbu zobrazte región štátu.

Čo viete povedať o centrálnej Európe? Koľko celkovej variancie pôvodných dát je zachovaných vo vizualizácii?